# Zach: the following 2 code cells are separate versions of the same thing. If you're planning on building off of it, start with the second cell, and reference the first one where appropriate. Comments on both are notes from meetings w/ Conroy & Secrest.

Useful resources:
*   [https://github.com/topics/lanczos-algorithm](https://github.com/topics/lanczos-algorithm)
*   [https://github.com/spartan-array/spartan/blob/master/spartan/examples/lanczos.py](https://github.com/spartan-array/spartan/blob/master/spartan/examples/lanczos.py)
*   [https://github.com/ecasiano/LanczosEigensolvers](https://github.com/ecasiano/LanczosEigensolvers)

In [1]:
'''
we expect alpha to be 1/2, so no need to loop

usually, we track how much the alpha value changes. if it changes by less than 0.01%, it will have converged and we can stop the loop.

no need to track beta value, it's just there for the ride

we expect alpha to change a lot at first, then slowly change less and less v quickly, should not be linear change

we want to generalize psi, rn psi is a Gaussian, we should have a normalization constant C times a function psi. generalize lanczos() to only take C and psi

0.5 is the ground state, should get 3.5, 5.5, 7.5, etc. from loop (excited states)

plot eigenvalues vs iterations 

in LaTeX: how/why does Lanczos algorithm work, and some pseudo code of what our algorithms do
'''

import sympy as sym
import numpy as np
import math

def gaussian(m, omega, hbar, x):
  return sym.exp(-(m * omega) / (2 * hbar) * (x ** 2))

def normalize_gauss(psi0, C, x):
  a, b = sym.solve((C ** 2) * sym.integrate((sym.Abs(psi0 / C) ** 2), (x, -sym.oo, sym.oo)) - 1, C)
  psi1 = b * (psi0 / C)
  return psi1, b

def hamiltonian(m, omega, hbar, psi1, x):
  return (-(hbar ** 2) / (2 * m)) * (sym.diff(psi1, x, 2)) + (0.5 * m * (omega ** 2) * (x ** 2) * psi1)

def tridiag(a, b, c, k1=-1, k2=0, k3=1):
  return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

def lanczos(m, omega, hbar, C, x):
  a, b = [], []
 
  psi0 = C * gaussian(m, omega, hbar, x)
  psi1, C = normalize_gauss(psi0, C, x)

  for i in range(1):
    w_prime = hamiltonian(m, omega, hbar, psi1, x)
    alpha1 = sym.integrate(psi1 * w_prime, (x, -sym.oo, sym.oo))
    a.append(alpha1)
    
    psi2 = w_prime - (alpha1 * psi1)
    beta2 = math.sqrt(sym.integrate((sym.Abs(psi2) ** 2), (x, -sym.oo, sym.oo)))
    b.append(beta2)

    psi1 = psi2/beta2

  return tridiag(a, b, a)


m, omega, hbar = 1, 1, 1
C, x = sym.Symbol('C'), sym.Symbol('x')

T = lanczos(m, omega, hbar, C, x)
print(T)

[[2.4822522981117435e-17
  2.48225229811174e-17 + 0.282094791773878*sqrt(pi)]
 [2.48225229811174e-17 + 0.282094791773878*sqrt(pi)
  2.4822522981117435e-17]]


In [2]:
'''
we expect alpha to change a lot at first, then slowly change less and less v quickly, should not be linear change

we want to generalize psi, rn psi is a Gaussian, we should have a normalization constant C times a function psi. generalize lanczos() to only take C and psi

0.5 is the ground state, should get 3.5, 5.5, 7.5, etc. from loop (excited states)

plot eigenvalues vs iterations 

in LaTeX: how/why does Lanczos algorithm work, and some pseudo code of what our algorithms do
'''

import sympy as sym
import numpy as np
import math

def gaussian(m, omega, hbar, x):
  return sym.exp(-(m * omega) / (2 * hbar) * (x ** 2))

def normalize_gauss(psi0, C, x):
  a, b = sym.solve((C ** 2) * sym.integrate((sym.Abs(psi0 / C) ** 2), (x, -sym.oo, sym.oo)) - 1, C)
  psi1 = b * (psi0 / C)
  return psi1, b

def hamiltonian(m, omega, hbar, psi1, x):
  return (-(hbar ** 2) / (2 * m)) * (sym.diff(psi1, x, 2)) + (0.5 * m * (omega ** 2) * (x ** 2) * psi1)

def tridiag(a, b, c, k1=-1, k2=0, k3=1):
  return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)

def lanczos(m, omega, hbar, C, x):
  a, b = [], []
 
  psi0 = C * gaussian(m, omega, hbar, x)
  psi1, C = normalize_gauss(psi0, C, x)

  #index, delta = 0, 0
  #while(True):
  for i in range (10):
    w_prime = hamiltonian(m, omega, hbar, psi1, x)
    alpha1 = sym.integrate(psi1 * w_prime, (x, -sym.oo, sym.oo))
    a.append(alpha1)
        
    psi2 = w_prime - (alpha1 * psi1)
    beta2 = math.sqrt(sym.integrate((sym.Abs(psi2) ** 2), (x, -sym.oo, sym.oo)))
    b.append(beta2)

    psi1 = psi2/beta2
    '''
    if index != 0:
        delta = a[index] / a[index - 1] * 100
        if delta <= 0.01:
            b.pop()
            return tridiag(b, a, b)

    index += 1
    '''
    return tridiag(b, a, b)
  

m, omega, hbar = 1, 1, 1
C, x = sym.Symbol('C'), sym.Symbol('x')

T = lanczos(m, omega, hbar, C, x)
print(T)
w = np.linalg.eig(T)
print(w)

[[0.282094791773878*sqrt(pi)
  2.48225229811174e-17 + 0.282094791773878*sqrt(pi)]
 [2.48225229811174e-17 + 0.282094791773878*sqrt(pi)
  0.282094791773878*sqrt(pi)]]


TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''